In [2]:
import pandas as pd
from openpyxl import load_workbook
import openpyxl
import pymysql
from tqdm import tqdm
import os

host = '218.155.74.39'
iid ='root'
pw = 'f1soft@170'
db_name = 'aihub_video_73'

conn = pymysql.connect(host=host, user= iid, password=pw, db=db_name, charset='utf8')

curs = conn.cursor(pymysql.cursors.DictCursor)

wb = load_workbook(filename = 'C:/Users/admin/Desktop/엑셀_양식/유형별_시간.xlsx', read_only = False, data_only = False)

wb.active.title = "시간 추출"

ws = wb.active

i = 1

for VEHICLE in tqdm(range(4)):

    for VIOLATION in range(2):
        i += 1

        for ACCIDENT_TYPE in range(1, 9):
            sql = f"""
                SELECT SUM(A.DURATION)
                FROM DATA A 
                    JOIN DATASET E
                    ON A.DATASET_ID = E.DATASET_ID
                    JOIN (
                        SELECT COUNT(A.FILENAME) AS CNT, A.FILENAME AS FN
                        FROM DATA A
                            JOIN
                            DATASET E
                            ON A.DATASET_ID = E.DATASET_ID
                        WHERE (A.CONFIRM_STATUS IN (0, 1, 2, 3, 4, 9))
                            AND (A.DATASET_ID BETWEEN 190 and 1281)
                            AND (E.TITLE LIKE ("%abies%"))
                        GROUP BY A.FILENAME
                    ) C
                    ON A.FILENAME = C.FN
                WHERE (A.CONFIRM_STATUS IN (0, 1, 2, 3, 4))
                    AND (A.DATASET_ID BETWEEN 190 and 1281)
                    AND (E.TITLE LIKE ("%abies%"))
                    AND (C.CNT = 1)
                    AND (A.FILENAME LIKE "%{VIOLATION}_{VEHICLE}_{ACCIDENT_TYPE}.mp4%")
            """

            curs.execute(sql)

            rows = curs.fetchall()

            ws.cell(i, ACCIDENT_TYPE + 2, rows[0]['SUM(A.DURATION)'])

wb.save('C:/Users/admin/Desktop/안불용_Time.xlsx')

100%|██████████| 4/4 [03:49<00:00, 57.46s/it]
